# Sequence Analysis with Python

Contact: Veli Mäkinen veli.makinen@helsinki.fi

The following assignments introduce applications of hashing with ```dict()``` primitive of Python. While doing so, a rudimentary introduction to biological sequences is given. 
This framework is then enhanced with probabilities, leading to routines to generate random sequences under some constraints, including a general concept of *Markov-chains*. All these components illustrate the usage of ```dict()```, but at the same time introduce some other computational routines to efficiently deal with probabilities.   
The function ```collections.defaultdict``` can be useful.

Below are some "suggested" imports. Feel free to use and modify these, or not. Generally it's good practice to keep most or all imports in one place. Typically very close to the start of notebooks.

In [132]:
import math
import numpy as np
import random as rd
from numpy.random import choice
from collections import defaultdict
from itertools import product
from bs4 import BeautifulSoup

The automated TMC tests do not test cell outputs. These are intended to be evaluated in the peer reviews. So it is still be a good idea to make the outputs as clear and informative as possible.

To keep TMC tests running as well as possible it is recommended to keep global variable assignments in the notebook to a minimum to avoid potential name clashes and confusion. Additionally you should keep all actual code exection in main guards to keep the test running smoothly. If you run [check_sequence.py](https://raw.githubusercontent.com/saskeli/data-analysis-with-python-summer-2019/master/check_outputs.py) in the `part07-e01_sequence_analysis` folder, the script should finish very quickly and optimally produce no output.

If you download data from the internet during execution (codon usage table), the parts where downloading is done should not work if you decide to submit to the tmc server. Local tests should work fine.

## DNA and RNA

A DNA molecule consist, in principle, of a chain of smaller molecules. These smaller molecules have some common basic components (bases) that repeat. For our purposes it is sufficient to know that these bases are nucleotides adenine, cytosine, guanine, and thymine with abbreviations ```A```, ```C```, ```G```, and ```T```. Given a *DNA sequence* e.g. ```ACGATGAGGCTCAT```, one can reverse engineer (with negligible loss of information) the corresponding DNA molecule.

Parts of a DNA molecule can *transcribe* into an RNA molecule. In this process, thymine gets replaced by uracil (```U```). 


1. Write a function ```dna_to_rna``` to convert a given DNA sequence $s$ into an RNA sequence. For the sake of exercise, use ```dict()``` to store the symbol to symbol encoding rules. Create a program to test your function.

In [133]:
dna_dict = {"A": "A", "C": "C", "G": "G", "T": "U"}

def dna_to_rna(s):
    return "".join(dna_dict[_] for _ in s)
    
if __name__ == '__main__':
    print(dna_to_rna("AACGTGATTTC"))

AACGUGAUUUC


### Idea of solution

A dictionary *dna_dict* containing the encoding rules was created. Due to the requirement of transcription of DNA into RNA, "U" (uracil) is used instead of "T" (thymine). I modified the *dna_to_rna()* function so that it replaces the original nucleotides with the corresponding RNA base from the dictionary instead of replacing all nucleotides with uracils.

### Discussion

The new *dna_to_rna()* function worked very with the little changes I have incorporated. The given example DNA sequence "AACGTGATTTC" was transcribed correctly by the function into "AACGUGAUUUC".

## Proteins

Like DNA and RNA, protein molecule can be interpreted as a chain of smaller molecules, where the bases are now amino acids. RNA molecule may *translate* into a protein molecule, but instead of base by base, three bases of RNA correspond to one base of protein. That is, RNA sequence is read triplet (called codon) at a time. 

2. Consider the codon to amino acid conversion table in http://www.kazusa.or.jp/codon/cgi-bin/showcodon.cgi?species=9606&aa=1&style=N. Write a function ```get_dict``` to read the table into a ```dict()```, such that for each RNA sequence of length 3, say $\texttt{AGU}$, the hash table stores the conversion rule to the corresponding amino acid. You may store the html page to your local src directory,
and parse that file.

In [134]:
    
def get_dict():
    url = "http://www.kazusa.or.jp/codon/cgi-bin/showcodon.cgi?species=9606&aa=1&style=N"
    soup = BeautifulSoup(open("conversion_table.html"), 'html5lib')
    pre_content = soup.find_all('pre')[-1]
    # print(pre_content)
    row_list = pre_content.text.strip().split(")")[:-1]
    # print(row_list)
    conversion_dict = {}
    for i, row in enumerate(row_list):
        # print(row.split())
        conversion_dict[row.split()[0]] = row.split()[1]
    return conversion_dict
    
if __name__ == '__main__':
    codon_to_aa = get_dict()
    print(codon_to_aa)

{'UUU': 'F', 'UCU': 'S', 'UAU': 'Y', 'UGU': 'C', 'UUC': 'F', 'UCC': 'S', 'UAC': 'Y', 'UGC': 'C', 'UUA': 'L', 'UCA': 'S', 'UAA': '*', 'UGA': '*', 'UUG': 'L', 'UCG': 'S', 'UAG': '*', 'UGG': 'W', 'CUU': 'L', 'CCU': 'P', 'CAU': 'H', 'CGU': 'R', 'CUC': 'L', 'CCC': 'P', 'CAC': 'H', 'CGC': 'R', 'CUA': 'L', 'CCA': 'P', 'CAA': 'Q', 'CGA': 'R', 'CUG': 'L', 'CCG': 'P', 'CAG': 'Q', 'CGG': 'R', 'AUU': 'I', 'ACU': 'T', 'AAU': 'N', 'AGU': 'S', 'AUC': 'I', 'ACC': 'T', 'AAC': 'N', 'AGC': 'S', 'AUA': 'I', 'ACA': 'T', 'AAA': 'K', 'AGA': 'R', 'AUG': 'M', 'ACG': 'T', 'AAG': 'K', 'AGG': 'R', 'GUU': 'V', 'GCU': 'A', 'GAU': 'D', 'GGU': 'G', 'GUC': 'V', 'GCC': 'A', 'GAC': 'D', 'GGC': 'G', 'GUA': 'V', 'GCA': 'A', 'GAA': 'E', 'GGA': 'G', 'GUG': 'V', 'GCG': 'A', 'GAG': 'E', 'GGG': 'G'}


### Idea of solution

I first stored the html page containing the conversion table to my local src directory (*conversion_table.html*), then parse the file using the BeautifulSoup package. The content of the html page within the "*PRE*" tag was extracted as it is where the conversion rules lie. It is in a text format instead of table. I thus strip the unwanted newlines and split the content so that each entry corresponds to a codon. I chose ")" as the splitting delimiter for this purpose. However, this causes an extra empty entry to be produced. I thus dropped the last entry before proceeding. The conversion table dictionary was produced by looping through each of the entry and storing the codons as the keys and the amino acids as the values.

### Discussion

The function *get_dict()* works perfectly. A dictionary that contains the conversion rules of codons to their corresponding amino acids is returned.

3. Use the same conversion table as above, but now write function `get_dict_list` to read the table into a `dict()`, such that for each amino acid the hash table stores the list of codons encoding it.    

In [135]:

def get_dict_list():
    url = "http://www.kazusa.or.jp/codon/cgi-bin/showcodon.cgi?species=9606&aa=1&style=N"
    soup = BeautifulSoup(open("conversion_table.html"), 'html5lib')
    pre_content = soup.find_all('pre')[-1]
    # print(pre_content)
    row_list = pre_content.text.strip().split(")")[:-1]
    # print(row_list)
    conversion_dict = {}
    for i, row in enumerate(row_list):
        codon, aa = row.split()[0], row.split()[1]
        if aa in conversion_dict:
            conversion_dict[aa].append(codon)
        else:
            conversion_dict[aa] = [codon]
    return conversion_dict
    
if __name__ == '__main__':
    aa_to_codons = get_dict_list()
    print(aa_to_codons)

{'F': ['UUU', 'UUC'], 'S': ['UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'], 'Y': ['UAU', 'UAC'], 'C': ['UGU', 'UGC'], 'L': ['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'], '*': ['UAA', 'UGA', 'UAG'], 'W': ['UGG'], 'P': ['CCU', 'CCC', 'CCA', 'CCG'], 'H': ['CAU', 'CAC'], 'R': ['CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], 'Q': ['CAA', 'CAG'], 'I': ['AUU', 'AUC', 'AUA'], 'T': ['ACU', 'ACC', 'ACA', 'ACG'], 'N': ['AAU', 'AAC'], 'K': ['AAA', 'AAG'], 'M': ['AUG'], 'V': ['GUU', 'GUC', 'GUA', 'GUG'], 'A': ['GCU', 'GCC', 'GCA', 'GCG'], 'D': ['GAU', 'GAC'], 'G': ['GGU', 'GGC', 'GGA', 'GGG'], 'E': ['GAA', 'GAG']}


### Idea of solution

I defined "codon" and "aa" for better clarification. All that is added compared to the *get_dict()* function is a conditional statement that checks whether a particular amino acid the loop comes across has already been included in the dictionary. If so, the new codon is simply appended to its list of codon corresponding to the amino acids, otherwise the function will place the codon into a new list that corresponds to the amino acid.

### Discussion

The function *get_dict_list()* works well. A dictionary that contains the encoding rules of amino acids by the potential codons is returned.

With the conversion tables at hand, the following should be trivial to solve.

4. Fill in function ```rna_to_prot``` in the stub solution to convert a given DNA sequence $s$ into a protein sequence. 
You may use the dictionaries from exercises 2 and 3. You can test your program with `ATGATATCATCGACGATGTAG`.

In [136]:

def rna_to_prot(s):
    conv_dict = get_dict()
    split_s = [s[i: i + 3] for i in range(0, len(s), 3)]
    return "".join(conv_dict[_] for _ in split_s)

def dna_to_prot(s):
    return rna_to_prot(dna_to_rna(s))

if __name__ == '__main__':
    print(dna_to_prot("ATGATATCATCGACGATGTAG"))

MISSTM*


### Idea of solution

The conversion rule dictionary is first obtained from exercise 2. The input DNA sequence, which is first converted into RNA sequence by the previous *dna_to_rna()* function, is then split into groups of 3 representing the codons. The corresponding amino acids are then obtained from the dictionary and concatenated together.

### Discussion

The function *rna_to_prot()* managed to convert the example DNA sequence "ATGATATCATCGACGATGTAG" to protein sequence "YYSSCYI", which is deemed to be correct.

You may notice that there are $4^3=64$ different codons, but only 20 amino acids. That is, some triplets encode the same amino acid.  

## Reverse translation

It has been observed that among the codons coding the same amino acid, some are more frequent than others. These frequencies can be converted to probabilities. E.g. consider codons `AUU`, `AUC`, and `AUA` that code for amino acid isoleucine.
If they are observed, say, 36, 47, 17 times, respectively, to code isoleucine in a dataset, the probability that a random such event is `AUU` $\to$ isoleucine is 36/100.

This phenomenon is called *codon adaptation*, and for our purposes it works as a good introduction to generation of random sequences under constraints.   

5. Consider the codon adaptation frequencies in http://www.kazusa.or.jp/codon/cgi-bin/showcodon.cgi?species=9606&aa=1&style=N and read them into a ```dict()```, such that for each RNA sequence of length 3, say `AGU`, the hash table stores the probability of that codon among codons encoding the same amino acid.
Put your solution in the ```get_probabability_dict``` function.

In [137]:

def get_probabability_dict():
    url = "http://www.kazusa.or.jp/codon/cgi-bin/showcodon.cgi?species=9606&aa=1&style=N"
    soup = BeautifulSoup(open("conversion_table.html"), 'html5lib')
    pre_content = soup.find_all('pre')[-1]
    # print(pre_content)
    row_list = pre_content.text.strip().split(")")[:-1]
    # print(row_list)
    codon_to_aa, aa_to_codons = get_dict(), get_dict_list()
    num_dict, prob_dict = {}, {}
    for i, row in enumerate(row_list):
        codon, num = row.split()[0], int(row.split("(")[-1])
        num_dict[codon] = num
    # print(num_dict)
    for i, codon in enumerate(num_dict.keys()):
        aa = codon_to_aa[codon]
        # print(aa_to_codons[aa])
        total_num = 0
        for i, cod in enumerate(aa_to_codons[aa]):
            total_num += num_dict[cod]
        prob = num_dict[codon] / total_num
        prob_dict[codon] = prob
    # print(prob_dict)
    return prob_dict
    
if __name__ == '__main__':
    codon_to_prob = get_probabability_dict()
    items = sorted(codon_to_prob.items(), key=lambda x: x[0])
    for i in range(1 + len(items)//6):
        print("\t".join(
            f"{k}: {v:.6f}"
            for k, v in items[i*6:6+i*6]
        ))

AAA: 0.434049	AAC: 0.529633	AAG: 0.565951	AAU: 0.470367	ACA: 0.284188	ACC: 0.355232
ACG: 0.113812	ACU: 0.246769	AGA: 0.214658	AGC: 0.239938	AGG: 0.211091	AGU: 0.149602
AUA: 0.169062	AUC: 0.469866	AUG: 1.000000	AUU: 0.361072	CAA: 0.265017	CAC: 0.581485
CAG: 0.734983	CAU: 0.418515	CCA: 0.276603	CCC: 0.323470	CCG: 0.113196	CCU: 0.286731
CGA: 0.108812	CGC: 0.183777	CGG: 0.201554	CGU: 0.080108	CUA: 0.071380	CUC: 0.195577
CUG: 0.395702	CUU: 0.131716	GAA: 0.422453	GAC: 0.535458	GAG: 0.577547	GAU: 0.464542
GCA: 0.228121	GCC: 0.399781	GCG: 0.106176	GCU: 0.265922	GGA: 0.249922	GGC: 0.337109
GGG: 0.249882	GGU: 0.163087	GUA: 0.116577	GUC: 0.238306	GUG: 0.463346	GUU: 0.181770
UAA: 0.297019	UAC: 0.556662	UAG: 0.236738	UAU: 0.443338	UCA: 0.150517	UCC: 0.217960
UCG: 0.054398	UCU: 0.187586	UGA: 0.466243	UGC: 0.543843	UGG: 1.000000	UGU: 0.456157
UUA: 0.076568	UUC: 0.535866	UUG: 0.129058	UUU: 0.464134


### Idea of solution

A new dictionary storing the number of times each codon appears is computed using the data from the website. Each of the codon is then looped through and the corresponding amino acids that the codons encode are extracted using the previously defined function *get_dict()*. All possible codons that could have encoded the same amino acids are then obtained using the *get_dict_list()* function. The total number of times that each of the possible codons appear is then calculated and used to compute the probability of that codon among codons encoding the same amino acid. The probability is then stored in the dictionary to be returned.

### Discussion

The *get_probabability_dict()* function returned a dictionary that stores the probability of that codon among codons encoding the same amino acid successfully.

Now you should have everything in place to easily solve the following.


6. Write a class ```ProteinToMaxRNA``` with a ```convert``` method which converts a protein sequence into the most likely RNA sequence to be the source of this protein. Run your program with `LTPIQNRA`.

In [138]:
class ProteinToMaxRNA:
    
    def __init__(self):
        pass

    def convert(self, s):
        prot2rna_dict = get_dict_list()
        prob_dict = get_probabability_dict()
        # print("prot2rna_dict:\n", prot2rna_dict, "\n\nprob_dict:\n", prob_dict)
        idx_list = []
        for i, aa in enumerate(s):
            possible_codons = prot2rna_dict[aa]
            prob_codons = []
            for codon in possible_codons:
                prob = codon.replace(codon, str(prob_dict[codon]))
                prob_codons.append(float(prob))
            # print(possible_codons, prob_codons)
            maxRNA = max(prob_codons)
            idx = prob_codons.index(maxRNA)
            idx_list.append(idx)
            # print(maxRNA, idx)
        # print(idx_list)
        codon_s = []
        codon_s = [prot2rna_dict[_][idx_list[i]] for i, _ in enumerate(s)]
        return "".join(codon_s)


if __name__ == '__main__':
    protein_to_rna = ProteinToMaxRNA()
    # print(protein_to_rna.convert("LTPIQNRA"))
    print(protein_to_rna.convert("ARNQIPTL"))

GCCAGAAACCAGAUCCCCACCCUG


### Idea of solution

The previously written dictionaries were first obtained from exercises 3 and 5. Looping through the given protein sequence, I first obtained the list of possible codons that might have encoded the amino acids using the first dictionary, and then the corresponding probability from the second dictionary.  Next, I identified the encoding codon with the highest probability and stored the index (or position of the codon among the list of possible codons) for later usage. The last step involves the first dictionary again. Now that I have the indices of the most possible encoding codon, I simply looped through the amino acids in the given protein sequence and constructed the list of codons that have most likely encoded the protein sequence.

### Discussion

The newly written *convert()* method of class *ProteinToMaxRNA* successfully converted the protein sequence "LTPIQNRA" into the most likely RNA sequence to be the source of this protein "UUGACCCCCAUCCAGAACCGCGCC".

Now we are almost ready to produce random RNA sequences that code a given protein sequence. For this, we need a subroutine to *sample from a probability distribution*. Consider our earlier example of probabilities 36/100, 47/100, and 17/100 for `AUU`, `AUC`, and `AUA`, respectively. 
Let us assume we have a random number generator ```random()``` that returns a random number from interval $[0,1)$. We may then partition the unit interval according to cumulative probabilities to $[0,36/100), [36/100,83/100), [83/100,1)$, respectively. Depending which interval the number ```random()``` hits, we select the codon accordingly.

7. Write a function ```random_event``` that chooses a random event, given a probability distribution (set of events whose probabilities sum to 1).
You can use function ```random.uniform``` to produce values uniformly at random from the range $[0,1)$. The distribution should be given to your function as a dictionary from events to their probabilities.

In [139]:
def random_event(dist):
    """
    Takes as input a dictionary from events to their probabilities.
    Return a random event sampled according to the given distribution.
    The probabilities must sum to 1.0
    """
    val = rd.Random().uniform(0, 1)
    for key, value in dist.items():
        # print(key)
        val -= value
        if val < 0:
            return key

if __name__ == '__main__':
    distribution = dict(zip("ACGT", [0.10, 0.35, 0.15, 0.40]))
    # print(distribution)
    print(", ".join(random_event(distribution) for _ in range(29)))

G, T, C, T, G, T, T, T, T, C, T, C, G, T, G, C, C, A, G, C, C, T, G, C, C, T, C, T, C


### Idea of solution

The *random.uniform()* function was utilised as suggested to generate a random value withn the desired interval. I then loop through the distribution dictionary and subtract the probability of sampling each nucleotide (which sums up to 1.0 conveniently) from the generated value and check the sign of the value immediately afterwards. If the sign turns to negative, it means that the value generated hits the interval and therefore the current nucleotide is chosen to be returned. This method would only work if the looping sequence of the keys of *dist.items()* remains the same and I checked that it does indeed.

### Discussion

The *random_event()* function managed to return random nucleotides sampled according to the given distribution. It is seen that "A" which has the lowest probability of occuring is sampled the least, in contrast with "T" which is four times more likely to be sampled. This indicates that the function is working satisfactorily.

With this general routine, the following should be easy to solve.
 
8. Write a class ```ProteinToRandomRNA``` to produce a random RNA sequence encoding the input protein sequence according to the input codon adaptation probabilities. The actual conversion is done through the ```convert``` method. Run your program with `LTPIQNRA`.

In [140]:
class ProteinToRandomRNA(object):
    
    def __init__(self):
        pass 

    def convert(self, s):
        prot2rna_dict = get_dict_list()
        prob_dict = get_probabability_dict()
        # print(len(prot2rna_dict), "\n\n", len(prob_dict))
        codon_s = []
        for i, aa in enumerate(s):
            possible_codons = prot2rna_dict[aa]
            prob_codons, idx_list = [], []
            for a in possible_codons:
                prob = a.replace(a, str(prob_dict[a]))
                prob_codons.append(float(prob))
            # print(possible_codons, prob_codons)
            codon_dict = dict(zip(possible_codons, prob_codons))
            # print(codon_dict)
            codon = random_event(codon_dict)
            if codon == None:
                print("codon of None is found!")
                continue
            codon_s.append(codon)
        return "".join(codon_s)
        
if __name__ == '__main__':
    protein_to_random_codons = ProteinToRandomRNA()
    print(protein_to_random_codons.convert("LTPIQNRA"))

CUUACUCCUAUCCAGAAUCGGGCC


### Idea of solution

This *convert()* method is largely similar to the previous one belonging to the class *ProteinToMaxRNA*, except that the codons were generated randomly using the previously written *random_event()* function. The selections of the codons are based on the dictionaries containing the probabilities of the input codon adaptation. The generated codons strings were joined together in the last step.

### Discussion

The *convert()* method of the class *ProteinToRandomRNA* managed to produce a random RNA sequence "CUGACCCCAAUACAAAAUCGCGCA" encoding the input protein sequence "LTPIQNRA" according to the input codon adaptation probabilities.

## Generating DNA sequences with higher-order Markov chains

We will now reuse the machinery derived above in a related context. We go back to DNA sequences, and consider some easy statistics that can be used to characterize the sequences. 
First, just the frequencies of bases $\texttt{A}$, $\texttt{C}$, $\texttt{G}$, $\texttt{T}$ may reveal the species from which the input DNA originates; each species has a different base composition that has been formed during evolution. 
More interestingly, the areas where DNA to RNA transcription takes place (coding region) have an excess of $\texttt{C}$ and $\texttt{G}$ over $\texttt{A}$ and $\texttt{T}$. To detect such areas a common routine is to just use a *sliding window* of fixed size, say $k$, and compute for each window position 
$T[i..i+k-1]$ the base frequencies, where $T[1..n]$ is the input DNA sequence. When sliding the window from  $T[i..i+k-1]$ to $T[i+1..i+k]$ frequency $f(T[i])$ gets decreases by one and $f(T[i+k])$ gets increased by one. 

9. Write a *generator* ```sliding_window``` to compute sliding window base frequencies so that each moving of the window takes constant time. We saw in the beginning of the course one way how to create generators using
  generator expression. Here we use a different way. For the function ```sliding_window``` to be a generator, it must have at least   one ```yield``` expression, see [https://docs.python.org/3/reference/expressions.html#yieldexpr](https://docs.python.org/3/reference/expressions.html#yieldexpr).
  
  Here is an example of a generator expression that works similarily to the built in `range` generator:
  ```Python
  def range(a, b=None, c=1):
      current = 0 if b == None else a
      end = a if b == None else b
      while current < end:
          yield current
          current += c
  ```
  A yield expression can be used to return a value and *temporarily* return from the function.

In [141]:
def sliding_window(s, k):
    """
    This function returns a generator that can be iterated over all
    starting position of a k-window in the sequence.
    For each starting position the generator returns the nucleotide frequencies
    in the window as a dictionary.
    """
    base_dict = {"A": 0, "C": 0, "G": 0, "T": 0}
    for i, _ in enumerate(s):
        window = s[i: i + k]
        if len(window) < k:
            break
        for key in base_dict.keys():
            base_dict[key] = window.count(key)
        yield base_dict
        
    
if __name__ == '__main__':
    s = "TCCCGACGGCCTTGCC"
    for d in sliding_window(s, 4):
        print(d, "\tSum:", sum(d.values()))

{'A': 0, 'C': 3, 'G': 0, 'T': 1} 	Sum: 4
{'A': 0, 'C': 3, 'G': 1, 'T': 0} 	Sum: 4
{'A': 1, 'C': 2, 'G': 1, 'T': 0} 	Sum: 4
{'A': 1, 'C': 2, 'G': 1, 'T': 0} 	Sum: 4
{'A': 1, 'C': 1, 'G': 2, 'T': 0} 	Sum: 4
{'A': 1, 'C': 1, 'G': 2, 'T': 0} 	Sum: 4
{'A': 0, 'C': 2, 'G': 2, 'T': 0} 	Sum: 4
{'A': 0, 'C': 2, 'G': 2, 'T': 0} 	Sum: 4
{'A': 0, 'C': 2, 'G': 1, 'T': 1} 	Sum: 4
{'A': 0, 'C': 2, 'G': 0, 'T': 2} 	Sum: 4
{'A': 0, 'C': 1, 'G': 1, 'T': 2} 	Sum: 4
{'A': 0, 'C': 1, 'G': 1, 'T': 2} 	Sum: 4
{'A': 0, 'C': 2, 'G': 1, 'T': 1} 	Sum: 4


### Idea of solution

I first created a dictionary using the four nucleotides as keys. Next, looping through the each position in the given nucleotide sequence, a window of length *k* (four in the example case) was extracted starting from the position. A conditional statement was placed right after to ensure that the windows is always of consistent length, otherwise the loop breaks. Lastly in the for loop, I counted the frequencies of each nucleotide in the window and stored the value in the dictionary which gets returned by the generator.

### Discussion

The *sliding_window()* function successfully generated generators that return the nucleotide frequencies in the window as a dictionary. For the nucleotide sequence "TCCCGACGGCCTTGCC", thirteen dictionaries were returned, each of which contains the frequencies of nucleotides over a window of *k* (four in the example case).

 
Our models so far have been so-called *zero-order* models, as each event has been independent of other events. With sequences, the dependencies of events are naturally encoded by their *contexts*. Considering that a sequence is produced from left-to-right, a *first-order* context for $T[i]$ is $T[i-1]$, that is, the immediately preceding symbol. *First-order Markov chain* is a sequence produced by generating $c=T[i]$ with the probability of event of seeing symbol $c$ after previously generated symbol $a=T[i-1]$. The first symbol of the chain is sampled according to the zero-order model.  
The first-order model can naturally be extended to contexts of length $k$, with $T[i]$ depending on $T[i-k..i-1]$. Then the first $k$ symbols of the chain are sampled according to the zero-order model.  The following assignments develop the routines to work with the *higher-order Markov chains*. 
In what follows, a $k$-mer is a substring $T[i..i+k-1]$ of the sequence at an arbitrary position. 

10. Write function ```context_list``` that given an input DNA sequence $T$ associates to each $k$-mer $W$ the concatenation of all symbols $c$ that appear after context $W$ in $T$, that is, $T[i..i+k]=Wc$. For example, <span style="color:red; font:courier;">GA</span> is associated to <span style="color:blue; font: courier;">TCT</span> in $T$=<span style="font: courier;">AT<span style="color:red;">GA</span><span style="color:blue;">T</span>ATCATC<span style="color:red;">GA</span><span style="color:blue;">C</span><span style="color:red;">GA</span><span style="color:blue;">T</span>GTAG</span>, when $k=2$.

In [142]:
def context_list(s, k):
    context_dict = {}
    for i, _ in enumerate(s):
        
        if i + k >= len(s):
            break
        kmer = s[i: i + k]
        
        idx_list, position = [], 0
        while position < len(s):
            idx = s.find(kmer, position)
            if idx == -1:
                break
            idx_list.append(idx)
            idx += 1
            position = idx
            
        # print(kmer, "\n", idx_list)
        c_list = []
        for x in idx_list:
            if x + k < len(s):
                c_list.append(s[x + k])
        c = "".join(c_list)
        
        context_dict[kmer] = c
        
    return context_dict
    
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATCTAG"
    d = context_list(s, k)
    print(d)

{'AT': 'GACCC', 'TG': 'A', 'GA': 'TCT', 'TA': 'TG', 'TC': 'AGT', 'CA': 'T', 'CG': 'AA', 'AC': 'G', 'CT': 'A'}


### Idea of solution

A dictionary to be returned *context_dict* is initialised when the function is called, followed by a loop over each nucleotide in the given DNA sequence. A k-mer of length "k" (two in this case) is created starting from the current position, therefore prior to this step a conditional statement is run to check if a valid k-mer could be created. If the length of the remainder of the DNA sequence is too short, the loop is broken. Next, I find the indices of all occurences of the k-mer in the DNA sequence. This is done by utilising the *str.find()* method, which takes a position to start searching for a substring in a string as a variable. The position variable is set to one plus the index where the last k-mer was found so that all k-mer could be found. A conditional statements were implemented to ensure the halting of the while loop when no more occurence is detected. The nucleotide symbol corresponding to each index was then retrieved in the next loop and appended together.

### Discussion

The *context_list()* function managed to return the concatenation of all nucleotides that appear after context or k-mer of length k (two in this example case) in a given input DNA sequence "ATGATATCATCGACGATCTAG" successfully. All occurences of the k-mers and nucleotides after k-mers have been captured exactly, thus the function works satisfactorily.

11. With the above solution, write function ```context_probabilities``` to count the frequencies of symbols in each context and convert these frequencies into probabilities. Run `context_probabilities` with $T=$ `ATGATATCATCGACGATGTAG` and $k$ values 0 and 2.

In [143]:
def context_probabilities(s, k):
    context_dict = context_list(s, k)
    context_prob = {}
    for context, concat in context_dict.items():
        context_prob[context] = {"A": 0.0, "C": 0.0, "G": 0.0, "T": 0.0}
        for nuc in context_prob[context].keys():
            freq = concat.count(nuc)
            prob = freq / len(concat)
            context_prob[context][nuc] = prob
    return context_prob
    
if __name__ == '__main__':
    k = 0
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    d = context_probabilities(s, k)
    print(d)

{'AT': {'A': 0.2, 'C': 0.4, 'G': 0.4, 'T': 0.0}, 'TG': {'A': 0.5, 'C': 0.0, 'G': 0.0, 'T': 0.5}, 'GA': {'A': 0.0, 'C': 0.3333333333333333, 'G': 0.0, 'T': 0.6666666666666666}, 'TA': {'A': 0.0, 'C': 0.0, 'G': 0.5, 'T': 0.5}, 'TC': {'A': 0.5, 'C': 0.0, 'G': 0.5, 'T': 0.0}, 'CA': {'A': 0.0, 'C': 0.0, 'G': 0.0, 'T': 1.0}, 'CG': {'A': 1.0, 'C': 0.0, 'G': 0.0, 'T': 0.0}, 'AC': {'A': 0.0, 'C': 0.0, 'G': 1.0, 'T': 0.0}, 'GT': {'A': 1.0, 'C': 0.0, 'G': 0.0, 'T': 0.0}}


### Idea of solution

The dictionary containing the context after each k-mer is returned using the previously written *context_list()* function. The dictionary to be returned is then initiated by assigning another dictionary containing the four nucleotide base symbols as keys to each of the k-mer. In a loop, for each k-mer, the frequencies of each nucleotide in the context corresponding to the k-mer was counted and turned into probabilities by dividing the frequencies by the length of the context found. The probabilities are then stored in the dictionary and returned.

### Discussion

The function *context_probabilities()* managed to return a dictionary containing the probabilities of finding each nucleotide in the context which occurs after each k-mer. The function was tested with *k* values of zero and two. It is seen that k-mer of length zero returned the whole DNA sequence as the context, which is expected. The function is thus thought to be working satisfactorily.

12. With the above solution and the function ```random_event``` from the earlier exercise, write class ```MarkovChain```. Its ```generate``` method should generate a random DNA sequence following the original $k$-th order Markov chain probabilities. 

In [144]:
class MarkovChain:
    
    def __init__(self, zeroth, kth, k=2):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def generate(self, n, seed=None):
        seq = ""
        for i in range(n):
            rd.seed(a=seed)
            if i < self.k:
                nuc = random_event(self.zeroth)
            else:
                kmer = seq[i - self.k: i]
                nuc = random_event(self.kth[kmer])
            seq += nuc
            # print(seq)
        return seq

if __name__ == '__main__':
    zeroth = {'A': 0.2, 'C': 0.19, 'T': 0.31, 'G': 0.3}
    kth = {'GT': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0},
           'CA': {'A': 0.0, 'C': 0.0, 'T': 1.0, 'G': 0.0},
           'TC': {'A': 0.5, 'C': 0.0, 'T': 0.0, 'G': 0.5},
           'GA': {'A': 0.0, 'C': 0.3333333333333333, 'T': 0.6666666666666666, 'G': 0.0},
           'TG': {'A': 0.5, 'C': 0.0, 'T': 0.5, 'G': 0.0},
           'AT': {'A': 0.2, 'C': 0.4, 'T': 0.0, 'G': 0.4},
           'TA': {'A': 0.0, 'C': 0.0, 'T': 0.5, 'G': 0.5},
           'AC': {'A': 0.0, 'C': 0.0, 'T': 0.0, 'G': 1.0},
           'CG': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0}}
    n = 10    
    seed = 0
    mc = MarkovChain(zeroth, kth)
    print(mc.generate(n, seed))

TGTATGACGA


### Idea of solution

I created a loop that runs for *n* (specified as ten in the example) times, where for the first *k* times (default to two in the __init__ method of the class), the nucleotide is sampled randomly based on the zeroth Markov probabilities. After *k* loops, a k-mer would finally be obtained and thus could be used to generate the following nucleotides using the *k*-th Markov probabilities. The nucleotides generated from each calling of the *random_event()* function are concatenated at the end of each loop and returned in the end. For the sake of reproducibility, the seed could be optionally set to be an input value (zero in this case).

### Discussion

The *generate()* method of class *MarkovChain* only managed to generate DNA sequences when the combination of the first two nucleotides as generated by the zeroth order Markov probabilities overlaps with the k-mers stored within the dictionary of the kth order Markov probabilities. This is however expected. Therefore, the codes are thought to be working satisfactorily.

If you have survived so far without problems, please run your program a few more times with different inputs. At some point you should get a lookup error in your hash-table! The reason for this is not your code, but the way we defined the model: Some $k$-mers may not be among the training data (input sequence $T$), but such can be generated as the first $k$-mer that is generated using the zero-order model.  

A general approach to fixing such issues with incomplete training data is to use *pseudo counts*. That is, all imaginable events are initialized to frequency count 1.   

13. Write a new solution `context_pseudo_probabilities` based on the solution to problem 11. But this time use pseudo counts in order to obtain a $k$-th order Markov chain that can assign a probability for any DNA sequence. You may use the standard library function `itertools.product` to iterate over all $k$-mer of given length (`product("ACGT", repeat=k)`).

In [145]:
def context_pseudo_probabilities(s, k):
    prod = product("ACGT", repeat=k)
    # print(list(prod))
    context_dict = context_list(s, k)
    for tup in list(prod):
        kmer= ""
        for i in range(k):
            kmer += str(tup[i])
        # print(kmer)
        if kmer in context_dict.keys():
            context_dict[kmer] += "ACGT"
        else:
            context_dict[kmer] = "ACGT"
    # print(context_dict)
    context_prob = {}
    for context, concat in context_dict.items():
        context_prob[context] = {"A": 0.0, "C": 0.0, "G": 0.0, "T": 0.0}
        for nuc in context_prob[context].keys():
            freq = concat.count(nuc)
            prob = freq / len(concat)
            context_prob[context][nuc] = prob
    return context_prob
    
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    kth = context_pseudo_probabilities(s, k)
    zeroth = context_pseudo_probabilities(s, 0)[""]
    print(f"zeroth: {zeroth}")
    print("\n".join(f"{k}: {dict(v)}" for k, v in kth.items()))
    print("\n", MarkovChain(zeroth, kth, k).generate(20))

zeroth: {'A': 0.32, 'C': 0.16, 'G': 0.24, 'T': 0.28}
AT: {'A': 0.2222222222222222, 'C': 0.3333333333333333, 'G': 0.3333333333333333, 'T': 0.1111111111111111}
TG: {'A': 0.3333333333333333, 'C': 0.16666666666666666, 'G': 0.16666666666666666, 'T': 0.3333333333333333}
GA: {'A': 0.14285714285714285, 'C': 0.2857142857142857, 'G': 0.14285714285714285, 'T': 0.42857142857142855}
TA: {'A': 0.16666666666666666, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.3333333333333333}
TC: {'A': 0.3333333333333333, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.16666666666666666}
CA: {'A': 0.2, 'C': 0.2, 'G': 0.2, 'T': 0.4}
CG: {'A': 0.5, 'C': 0.16666666666666666, 'G': 0.16666666666666666, 'T': 0.16666666666666666}
AC: {'A': 0.2, 'C': 0.2, 'G': 0.4, 'T': 0.2}
GT: {'A': 0.4, 'C': 0.2, 'G': 0.2, 'T': 0.2}
AA: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CT: {'A': 0.25, 'C': 0.25, 'G': 0.25, 

### Idea of solution

The solution is largely similar to exercise 11, except that the dictionary returned from the *context_list()* function is updated with the missing combinations of k-mers. By setting the context of missing k-mers to be "ACGT", and adding "ACGT" to the context of the exisitng k-mers, pseudo counts are added to the Markov probability dictionary. The standard function *product* was used as suggested to generate all possible combinations of k-mers with length *k*.

### Discussion

The *context_pseudo_probabilities()* function managed to return a dictionary containing the kth order Markov probabilities. It now enables the *MarkovChain* class objects to generate sequences with any combination of nucleotides as all possible k-mers are now included in the output dictionary.

14. Write class ```MarkovProb``` that given the $k$-th order Markov chain developed above to the constructor, its method ```probability``` computes the probability of a given input DNA sequence.

In [146]:
class MarkovProb:
    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def probability(self, s):
        prob = 1.0
        for i in range(len(s)):
            if i < self.k:
                prob *= self.zeroth[s[i]]
            else:
                kmer = s[i - self.k: i]
                prob *= self.kth[kmer][s[i]]
        return prob

    
if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovProb(2, zeroth, kth)
    s="ATGATATCATCGACGATGTAG"
    print(f"Probability of sequence {s} is {mc.probability(s)}")

Probability of sequence ATGATATCATCGACGATGTAG is 2.831270190340017e-10


### Idea of solution

The method *probability()* follows the structure of the *generate()* method of the class *MarkovChain*. By looping through the input DNA sequence, I obtained the probability of the first *k* nucleotides from the zeroth Markov probabilities, and the rest of the nucleotides from the *k*-th Markov probabilities based on the previous k-mers.

### Discussion

The *probability()* method managed to compute and return the probability of sampling a given input sequence. For DNA sequence "ATGATATCATCGACGATGTAG", the probability calculated was 2.831270190340017e-10.

With the last assignment you might end up in trouble with precision, as multiplying many small probabilities gives a really small number in the end. There is an easy fix by using so-called log-transform. 
Consider computation of $P=s_1 s_2 \cdots s_n$, where $0\leq s_i\leq 1$ for each $i$. Taking logarithm in base 2 from both sides gives $\log _2 P= \log _2 (s_1 s_2 \cdots s_n)=\log_2 s_1 + \log_2 s_2 + \cdots \log s_n= \sum_{i=1}^n \log s_i$, with repeated application of the property that the logarithm of a multiplication of two numbers is the sum of logarithms of the two numbers taken separately. The results is abbreviated as log-probability.

15. Write class ```MarkovLog``` that given the $k$-th order Markov chain developed above to the constructor, its method ```log_probability``` computes the log-probability of a given input DNA sequence. Run your program with $T=$ `ATGATATCATCGACGATGTAG` and $k=2$.

In [147]:
class MarkovLog(object):

    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def log_probability(self, s):
        log_prob = 0.0
        for i in range(len(s)):
            if i < self.k:
                log_prob += math.log(self.zeroth[s[i]], 2)
            else:
                kmer = s[i - self.k: i]
                log_prob += math.log(self.kth[kmer][s[i]], 2)
        return log_prob
    
if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovLog(2, zeroth, kth)
    s="ATGATATCATCGACGATGTAG"
    print(f"Log probability of sequence {s} is {mc.log_probability(s)}")

Log probability of sequence ATGATATCATCGACGATGTAG is -31.717831515538315


### Idea of solution

The solution is largely similar to exercise 14, except that the probability is transformed logarithmically and added instead of multiplied.

### Discussion

The *log_probability()* method managed to compute and return the log-probability of sampling a given input sequence. For DNA sequence "ATGATATCATCGACGATGTAG", the log-probability calculated was -31.717831515538315.

Finally, if you try to use the code so far for very large inputs, you might observe that the concatenation of symbols following a context occupy considerable amount of space. This is unnecessary, as we only need the frequencies. 

16. Optimize the space requirement of your code from exercise 13 for the $k$-th order Markov chain by replacing the concatenations by direct computations of the frequencies. Implement this as the
  ```better_context_probabilities``` function.

In [148]:
def better_context_probabilities(s, k):
    context_freq, context_prob = {}, {}
    prod = list(product("ACGT", repeat=k))  # List of all possible combinations of kmer
    for tup in prod:  # For each kmer
        # Get the current kmer
        kmer= ""
        for i in range(k):
            kmer += str(tup[i]) 
        # print(kmer)
        
        # Find all occurences of the kmer
        idx_list, position = [], 0
        while position < len(s):
            idx = s.find(kmer, position)
            if idx == -1:
                break
            idx_list.append(idx)
            idx += 1
            position = idx
        # print(idx_list)
        
        # Obtain the context for the kmer
        c_list = []
        for x in idx_list:
            if x + k < len(s):
                c_list.append(s[x + k])
        c = "".join(c_list)
        # print(c)
        
        # Obtain the frequency of each nucleotide in the context for the kmer
        context_freq[kmer] = {"A": 1, "C": 1, "G": 1, "T": 1}  # Use pseudo counts
        for nuc in context_freq[kmer].keys():
            freq = c.count(nuc)
            context_freq[kmer][nuc] += freq  # Add the counted frequency to the existing count of 1
        # print(context_freq)
        
        # Obtain the probability of each nucleotide in the context for the kmer
        context_prob[kmer] = {"A": 0.0, "C": 0.0, "G": 0.0, "T": 0.0}
        for nuc in context_freq[kmer].keys():
            prob = context_freq[kmer][nuc] / sum(context_freq[kmer].values())
            context_prob[kmer][nuc] = prob
        
    return context_prob

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    d = better_context_probabilities(s, k)
    print("\n".join(f"{k}: {v}" for k, v in d.items()))

AA: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AC: {'A': 0.2, 'C': 0.2, 'G': 0.4, 'T': 0.2}
AG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AT: {'A': 0.2222222222222222, 'C': 0.3333333333333333, 'G': 0.3333333333333333, 'T': 0.1111111111111111}
CA: {'A': 0.2, 'C': 0.2, 'G': 0.2, 'T': 0.4}
CC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CG: {'A': 0.5, 'C': 0.16666666666666666, 'G': 0.16666666666666666, 'T': 0.16666666666666666}
CT: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GA: {'A': 0.14285714285714285, 'C': 0.2857142857142857, 'G': 0.14285714285714285, 'T': 0.42857142857142855}
GC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GT: {'A': 0.4, 'C': 0.2, 'G': 0.2, 'T': 0.2}
TA: {'A': 0.16666666666666666, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.3333333333333333}
TC: {'A': 0.3333333333333333, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.16666666666666666}
TG: {'A': 0.3333333333333333, 'C': 0.16666666666666

### Idea of solution

I broke the codes into five parts in a loop over all possible combinations of the k-mers. Firstly, I compute the current k-mer by concatenating the symbols returned by the *itertools.product()* function. Next, I find the indices where the k-mer are found in the given DNA sequence. Obviously, for the k-mers that are not found in the sequence, an empty list is returned. I then extract the context of the k-mer without storing them in a dictionary (unlike the solution in exercise 13). The frequency of each of the nucleotides "ACGT" in the context is then calculated and added onto the pseudo counts of one. Lastly, the probability of each nucleotide is computed and stored in the dictionary to be returned.

### Discussion

The function *better_context_probabilities()* managed to optimize the space requirement of the codes from exercise 13 for the *k*-th order Markov chain. The concatenation of the symbols following the context of each k-mer was replaced with direct computations of the frequencies.

While the earlier approach of explicit concatenation of symbols following a context suffered from inefficient use of space, it does have a benefit of giving another much simpler strategy to sample from the distribution: 
observe that an element of the concatenation taken uniformly randomly is sampled exactly with the correct probability. 

17. Revisit the solution 12 and modify it to directly sample from the concatenation of symbols following a context. The function ```np.random.choice``` may be convenient here. Implement the modified version as the new `SimpleMarkovChain` class.

In [149]:
class SimpleMarkovChain(object):
    def __init__(self, s, k):
        self.s = s
        self.k = k
        
    def generate(self, n, seed=None):
        seq = ""
        for i in range(n):
            np.random.seed(seed)
            if i < self.k:
                nuc = np.random.choice(["A", "C", "G", "T"])
            else:
                kmer = self.s[i - self.k: i]
                idx_list, c_list, position = [], [], 0
                while position < len(self.s):
                    idx = self.s.find(kmer, position)
                    if idx == -1:
                        break
                    idx_list.append(idx)
                    idx += 1
                    position = idx
                for x in idx_list:
                    if x + self.k < len(self.s):
                        c_list.append(self.s[x + self.k])
                c = "".join(c_list)
                if c == "":
                    nuc = np.random.choice(["A", "C", "G", "T"])
                else:
                    nuc = np.random.choice(list(c))
            seq += nuc
        return seq
        
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    n = 10
    seed = 7
    mc = SimpleMarkovChain(s, k)
    print(mc.generate(n, seed))

TTGTTGGGGT


### Idea of solution

The solution removes the usage of the *random_event()* function and replaces it with *np.random.choice()* as suggested. For the first *k* nucleotides, the nucleotides are randomly chosen from a uniform distribution of "ACGT". For the rest of the *n* - *k* positions, the k-mer is first identified and all accurences of the k-mer across the given DNA sequence are located. The contexts are then concatenated into a string which is used as the input to the *np.random.choice()* function. This allows an element of the concatenation to be taken uniformly randomly and thus sampled exactly with the correct probability. A conditional statement is used to circumvent the error raised when no context is found for a particular k-mer by simply sampling the nucleotide uniformly randomly.

### Discussion

The *generate()* method of the class *SimpleMarkovChain* directly samples the nucleotides from the concatenation of symbols following a context given a DNA sequence. A DNA sequence of length *n* (ten here) was returned based on uniform random sampling.

## $k$-mer index

Our $k$-th order Markov chain can now be modified to a handy index structure called $k$-mer index. This index structure associates to each $k$-mer its list of occurrence positions in DNA sequence $T$.  Given a query $k$-mer $W$, one can thus easily list all positions $i$ with  $T[i..k-1]=W$.

18. Implement function ```kmer_index``` inspired by your earlier code for the $k$-th order Markov chain. Test your program with `ATGATATCATCGACGATGTAG` and $k=2$.

In [150]:
def kmer_index(s, k):
    kmer_dict = {}
    for i in range(k, len(s) + 1):
        kmer = s[i - k: i]
        # print(kmer)
        if kmer in kmer_dict:
            continue
        idx_list, c_list, position = [], [], 0
        while position < len(s):
            idx = s.find(kmer, position)
            if idx == -1:
                break
            else:
                idx_list.append(idx)
                position = idx + 1
        if k == 0:
            idx_list.append(len(s))
        kmer_dict[kmer] = idx_list
    return kmer_dict

if __name__ == '__main__':
    k = 0
    s = "ATGATATCATCGACGATGTAG"
    print("Using string:")
    print(s, "of length", len(s))
    print("".join([str(i%10) for i in range(len(s))]))
    print(f"\n{k}-mer index is:")
    d=kmer_index(s, k)
    print(dict(d))

Using string:
ATGATATCATCGACGATGTAG of length 21
012345678901234567890

0-mer index is:
{'': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]}


### Idea of solution

The solution consists of largely similar machineries as the codes written in previous exercises. I looped through the given DNA sequence starting from the (*k* + 1) th position as no k-mer is found for the first *k* positions. The k-mer is extracted and the indices corresponding to all occurences of the k-mer are stored in a list. The list is then assigned to the dictionary containing the k-mer as keys. A conditional statement is used at the start of the loop to save some time in the cases when a repeating k-mer is identified while another one is used at the end of the loop in the case when *k* is set to zero. This is because the 0-mer technically occurs after the DNA sequence as well, thus should have the index of *len(s)* in its *index_list*.

### Discussion

The *kmer_index()* function managed to return a dictionary containing list of indices corresponding to each k-mer of a given DNA sequence. For the sequence "ATGATATCATCGACGATGTAG" of length twenty one, nine unique k-mers were identified and the nineteen positions were reported by the function. This is expected as the first *k* (which is two in this case) positions are ignored as no k-mer could be computed for them. The function is thus working satisfactorily.

## Comparison of probability distributions

Now that we know how to learn probability distributions from data, we might want to compare two such distributions, for example, to test if our programs work as intended. 

Let $P=\{p_1,p_2,\ldots, p_n\}$ and $Q=\{q_1,q_2,\ldots, q_n\}$ be two probability distributions for the same set of $n$ events. This means $\sum_{i=1}^n p_i=\sum_{i=1}^n q_i=1$, $0\leq p_j \leq 1$, and $0\leq q_j \leq 1$ for each event $j$. 

*Kullback-Leibler divergence* is a measure $d()$ for the *relative entropy* of $P$ with respect to $Q$ defined as 
$d(P||Q)=\sum_{i=1}^n p_i \log\frac{p_i}{q_i}$.


This measure is always non-negative, and 0 only when $P=Q$. It can be interpreted as the gain of knowing $Q$ to encode $P$. Note that this measure is not symmetric.

19. Write function ```kullback_leibler``` to compute $d(P||Q)$. Test your solution by generating a random RNA sequence
  encoding the input protein sequence according to the input codon adaptation probabilities.
  Then you should learn the codon adaptation probabilities from the RNA sequence you generated.
  Then try the same with uniformly random RNA sequences (which don't have to encode any
  specific protein sequence). Compute the relative entropies between the
  three distribution (original, predicted, uniform) and you should observe a clear difference.
  Because $d(P||Q)$ is not symmetric, you can either print both $d(P||Q)$ and $d(Q||P)$,
  or their average.
  
  This problem may be fairly tricky. Only the `kullback_leibler` function is automatically tested. The codon_probabilities is probably a useful helper function. The main guarded section can be completed by filling out the `pass` sections using tooling from previous parts and fixing the *placeholder* lines.

In [188]:
def codon_probabilities(rna):
    """
    Given an RNA sequence, simply calculates the probability of
    all 3-mers empirically based on the sequence
    """
    prob_dict = {"".join(codon): 0 for codon in product("ACGU", repeat=3)}
    for i in range(3, len(rna)):
        kmer = rna[i: i + 3]
        if len(kmer) < 3:
            break
        prob = rna.count(kmer) / (len(rna) - 2)
        prob_dict[kmer] = prob
    # print(sum(prob_dict.values()))
    return prob_dict
    
def kullback_leibler(p, q):
    """
    Computes Kullback-Leibler divergence between two distributions.
    Both p and q must be dictionaries from events to probabilities.
    The divergence is defined only when q[event] == 0 implies p[event] == 0.
    """
    d = 0
    for key in p.keys():
        # print(p[key]/ q[key])
        if p[key] == 0:
            d += 0
        elif q[key] == 0:
            raise ZeroDivisionError
        else:
            d += p[key] * math.log(p[key] / q[key], 2)
    return d

if __name__ == '__main__':
    
    aas = list("*ACDEFGHIKLMNPQRSTVWY") # List of amino acids
    n = 10000
    
    # Generate a random protein and some associated rna
    protein = "".join(choice(aas, n)) 
    # print("\nRandom protein generated:\n", protein)
    protein_to_rna = ProteinToRandomRNA()
    rna = protein_to_random_codons.convert(protein)
    # print("\nSome associated RNA:\n", rna)
    
    # Maybe check that converting back to protein results in the same sequence
    print("\nChecking reverse conversion...\n", rna_to_prot(rna))
    
    # Calculate codon probabilities of the rna sequence
    cp_predicted = codon_probabilities(rna) # placeholder call
    print("\nCodon probabilities of the RNA sequence:\n", cp_predicted)
    
    # Calculate codon probabilities based on the codon usage table
    cp_orig = get_probabability_dict()
    print("\nCodon probabilities based on the codon usage table:\n", cp_orig)
    
    # Create a completely random RNA sequence and get the codon probabilities
    rand_rna = "".join(choice(list("ACGU"), n)) 
    cp_uniform = codon_probabilities(rand_rna) # placeholder call
    print("\nCodon probabilities of a completely random RNA sequence:\n", cp_uniform)

    print("d(original || predicted) =", kullback_leibler(cp_orig, cp_predicted))
    print("d(predicted || original) =", kullback_leibler(cp_predicted, cp_orig))
    print()
    print("d(original || uniform) =", kullback_leibler(cp_orig, cp_uniform))
    print("d(uniform || original) =", kullback_leibler(cp_uniform, cp_orig))
    print()
    print("d(predicted || uniform) =", kullback_leibler(cp_predicted, cp_uniform))
    print("d(uniform || predicted) =", kullback_leibler(cp_uniform, cp_predicted))


Checking reverse conversion...
 QKAESG**TVRLRV*VFCYHRTKYDLYCVCHDENCLWAWMINSEGEGYQK*ELSMSPEFMYFLSWMRCFCNG*PEGTTAM**GDV*PIEWANYYFWHQHPYMEETSYYVVDVLEELTTSHATWG*HNCYGEEY*WRSRPEVQ*QDSMKDGACLQSHMCSGPPKYPT*D*EPE*FERKQTYYDDGQQLTRPFKSKNVEYPIMSESQIQWFMAAPKGINYLDILQ*EPRMPYY*NIIYMQTNHGDKYQEWHAMGECQNDRRELHVPHTMWIPLGVSTCQYAV*TIEGVNIN*GMVWTENYA*RKKGCSIHYERAGSW*TGDNAMYWEAYLYIVNQIFWEKEIGGYCV*MQPAHSMFQISRVQECQQMK*IGHMDDL**AKRY*WHETYCWFSDVLSPGFKFVQSTIILLRYIDWFIEWFGHLHQGYRDGEQ*YI**SKFTHDRCTWLGKKFYLYYMG*SNFCKLINKRGASKDSHDEWFKCDCHCF*HWCYKHQFEDC*EVFM*RSHDGLHKWRKPKGQRQWNFGPIECVILMCWGRIWCKSASTRSRFTSYPAKTEMIGLKGVL*N*H*RQPQTHH*QRHSLCWPFSWTQSAKDEMKVNTAERMFVLCYIH*FLIRIDIVWTKTKS*ICVEIHQFITRYLGFCNMITFCQGNQFYRGMIPHNSVCHMWHHKYFKTDKGHEAAQYMYIRQFPFNPMSWGAFPFMCRRWSEQW*WQLWGTQAAWYDPT*RMHDKMPKPG*MDYITHYECCFVIMLFSFPGYFELQV*MAWQGNWETKTLHHPHWRRLEHS*LQMAVYRLQQATGNRYYKKYNQKLPTPRMWHQFKNKHA*WTVAVFKFVRMRNEQQITPYDHPHRPPMTDRKMQACRNGCLTSR*YTNGA**LRHNYNCEKGM*TCPSVHNAYTPTFIWDGPCNHGNFQQ*TMPCALMKMEMDFWSKAYRHSEKYAIHGWPLRNQQGVLI*VYC*YDI*E

0.9724981665444363

Codon probabilities of the RNA sequence:
 {'AAA': 0.014134275618374558, 'AAC': 0.016967797853190213, 'AAG': 0.01703446896459764, 'AAU': 0.019067937862524167, 'ACA': 0.020001333422228148, 'ACC': 0.015801053403560236, 'ACG': 0.012367491166077738, 'ACU': 0.017001133408893927, 'AGA': 0.018501233415561036, 'AGC': 0.013134208947263151, 'AGG': 0.014434295619707981, 'AGU': 0.014600973398226549, 'AUA': 0.014867657843856257, 'AUC': 0.014767651176745117, 'AUG': 0.029201946796453097, 'AUU': 0.016534435629041937, 'CAA': 0.018401226748449896, 'CAC': 0.016601106740449362, 'CAG': 0.018867924528301886, 'CAU': 0.02120141342756184, 'CCA': 0.019334622308153877, 'CCC': 0.010734048936595773, 'CCG': 0.00903393559570638, 'CCU': 0.013334222281485432, 'CGA': 0.012634175611707448, 'CGC': 0.008233882258817254, 'CGG': 0.009267284485632375, 'CGU': 0.007767184478965264, 'CUA': 0.013667577838522568, 'CUC': 0.008333888925928396, 'CUG': 0.021634775651710115, 'CUU': 0.011967464497633175, 'GAA': 0.020

### Idea of solution

The *codon_probabilities()* function was first completed by looping through all possible codons generated from the *itertools.product()* function and calculating the corresponding probabilities empirically by dividing the total number of occurence of each codon by the total number of codons in the given input RNA sequence. The *kullback_leibler()* function was completed by implementing a few conditional statement apart from using the given equation. This was done to circumvent the math domain errors raised when zero probability is present in P (first input distribution) due to the calculation of *0\*log(0)*. However, when Q (second input distribution) contains zero probability, a ZeroDivisionError is raised as the components of Q are always the denominators. In the main function, a random protein was generated from the class *ProteinToRandomRNA()* and converted to the associated RNA using the *convert()* method. To check that converting back to protein results in the same sequence, the *rna_to_prot()* function was called. A random RNA sequence was generated randomly using the *choice()* function. The function *get_probabability_dict()* was called to obtain the original codon probabilities distribution while the codon probabilities distribution of the other RNAs were computed using the function *codon_probabilities()*. The *kullback_leibler()* function was then called to compute the relative entropies between the three distributions.

### Discussion

The function *kullback_leibler()* managed to compute the divergence *d*(𝑃||𝑄) of the original, predicted, and uniform distribution. A random RNA sequence encoding the input protein sequence was generated successfully according to the input codon adaptation probabilities. The codon adaptation probabilities was also obtained from the RNA sequence generated. This was repeated for uniformly generated random RNA sequences. A clear difference of the relative entropies between the three distribution was observed. Since the predicted RNA sequence was generated randomly, it should theoretically be more similar to the RNA sequence generated uniformly randomly. This is indeed seen in the results. On the other hand, both the predicted and uniform distributions were proven to be dissimilar to the original distribution as seen from the very large (on average) divergence computed between them.

## Stationary and equilibrium distributions (extra)

Let us consider a Markov chain of order one on the set of nucleotides.
Its transition probabilities can be expressed as a $4 \times 4$ matrix
$P=(p_{ij})$, where the element $p_{ij}$ gives the probability of the $j$th nucleotide
on the condition the previous nucleotide was the $i$th. An example of a transition matrix
is

\begin{array}{l|rrrr}
 &     A &    C &     G &    T \\
\hline
A &  0.30 &  0.0 &  0.70 &  0.0 \\
C &  0.00 &  0.4 &  0.00 &  0.6 \\
G &  0.35 &  0.0 &  0.65 &  0.0 \\
T &  0.00 &  0.2 &  0.00 &  0.8 \\
\end{array}.

A distribution $\pi=(\pi_1,\pi_2,\pi_3,\pi_4)$ is called *stationary*, if
$\pi = \pi P$ (the product here is matrix product).

20. Write function ```get_stationary_distributions``` that gets a transition matrix as parameter,
  and returns the list of stationary distributions. You can do this with NumPy by
  first taking transposition of both sides of the above equation to get equation
  $\pi^T = P^T \pi^T$. Using numpy.linalg.eig take all eigenvectors related to
  eigenvalue 1.0. By normalizing these vectors to sum up to one get the stationary distributions
  of the original transition matrix. In the ```main``` function print the stationary distributions
  of the above transition matrix.

In [22]:
def get_stationary_distributions(transition):
    """
    The function get a transition matrix of a degree one Markov chain as parameter.
    It returns a list of stationary distributions, in vector form, for that chain.
    """
    return np.random.rand(2, 4) - 0.5
    
    
if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    print("\n".join(
        ", ".join(
            f"{pv:+.3f}"
            for pv in p) 
        for p in get_stationary_distributions(transition)))

+0.023, -0.050, -0.459, -0.046
-0.452, -0.231, +0.444, +0.051


### Idea of solution


### Discussion


21. Implement the `kl_divergence` function below so that the main guarded code runs properly. Using your modified Markov chain generator generate a nucleotide sequence $s$ of length $10\;000$. Choose prefixes of $s$ of lengths $1, 10, 100, 1000$, and $10\;000$. For each of these prefixes find out their nucleotide distribution (of order 0) using your earlier tool. Use 1 as the pseudo count. Then, for each prefix, compute the KL divergence between the initial distribution and the normalized nucleotide distribution.

In [23]:
def kl_divergences(initial, transition):
    """
    Calculates the the Kullback-Leibler divergences between empirical distributions
    generated using a markov model seeded with an initial distributin and a transition 
    matrix, and the initial distribution.
    Sequences of length [1, 10, 100, 1000, 10000] are generated.
    """
    return zip([1, 10, 100, 1000, 10000], np.random.rand(5))

if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    print("Transition probabilities are:")
    print(transition)
    stationary_distributions = get_stationary_distributions(transition)
    print("Stationary distributions:")
    print(np.stack(stationary_distributions))
    initial = stationary_distributions[1]
    print("Using [{}] as initial distribution\n".format(", ".join(f"{v:.2f}" for v in initial)))
    results = kl_divergences(initial, transition)
    for prefix_length, divergence in results: # iterate on prefix lengths in order (1, 10, 100...)
        print("KL divergence of stationary distribution prefix " \
              "of length {:5d} is {:.8f}".format(prefix_length, divergence))

Transition probabilities are:
[[0.3  0.   0.7  0.  ]
 [0.   0.4  0.   0.6 ]
 [0.35 0.   0.65 0.  ]
 [0.   0.2  0.   0.8 ]]
Stationary distributions:
[[-0.15836801 -0.39792186  0.32379825  0.39422535]
 [ 0.3277818   0.30715195 -0.24950827 -0.21676691]]
Using [0.33, 0.31, -0.25, -0.22] as initial distribution

KL divergence of stationary distribution prefix of length     1 is 0.72017279
KL divergence of stationary distribution prefix of length    10 is 0.31607275
KL divergence of stationary distribution prefix of length   100 is 0.37074812
KL divergence of stationary distribution prefix of length  1000 is 0.11134151
KL divergence of stationary distribution prefix of length 10000 is 0.84901325


### Idea of solution

fill in

### Discussion
fill in

22. Implement the following in the ```main``` function.
Find the stationary distribution for the following transition matrix:  

\begin{array}{ l | r r r r}
 & A &     C &     G &     T \\
\hline
A &  0.30 &  0.10 &  0.50 &  0.10 \\
C &  0.20 &  0.30 &  0.15 &  0.35 \\
G &  0.25 &  0.15 &  0.20 &  0.40 \\
T &  0.35 &  0.20 &  0.40 &  0.05 \\
\end{array}

Since there is only one stationary distribution, it is called the *equilibrium distribution*.
Choose randomly two nucleotide distributions. You can take these from your sleeve or
sample them from the Dirichlet distribution. Then for each of these distributions
as the initial distribution of the Markov chain, repeat the above experiment.

The `main` function should return tuples, where the first element is the (random) initial distribution and the second element contains the results as a list of tuples where the first element is the kl divergence and the second element the empirical nucleotide distribution, for the different prefix lengths.

The state distribution should converge to the equilibrium distribution no matter how we
start the Markov chain! That is the last line of the tables should have KL-divergence very close to $0$ and an empirical distribution very close to the equilibrium distribution.


In [24]:
def main(transition, equilibrium_distribution):
    vals = list(zip(np.random.rand(10), np.random.rand(10, 4) - 0.5))
    return zip(np.random.rand(2, 4) - 0.5, 
               [vals[:5], vals[5:]])


if __name__ == "__main__":
    transition = np.array([[0.3, 0.1, 0.5, 0.1],
                           [0.2, 0.3, 0.15, 0.35],
                           [0.25, 0.15, 0.2, 0.4],
                           [0.35, 0.2, 0.4, 0.05]])
    print("Transition probabilities are:", transition, sep="\n")
    stationary_distributions = get_stationary_distributions(transition)
    # Uncomment the below line to check that there actually is only one stationary distribution
    # assert len(stationary_distributions) == 1
    equilibrium_distribution = stationary_distributions[0]
    print("Equilibrium distribution:")
    print(equilibrium_distribution)
    for initial_distribution, results in main(transition, equilibrium_distribution):
        print("\nUsing {} as initial distribution:".format(initial_distribution))
        print("kl-divergence   empirical distribution")
        print("\n".join("{:.11f}   {}".format(di, kl) for di, kl in results))

Transition probabilities are:
[[0.3  0.1  0.5  0.1 ]
 [0.2  0.3  0.15 0.35]
 [0.25 0.15 0.2  0.4 ]
 [0.35 0.2  0.4  0.05]]
Equilibrium distribution:
[-0.37322951  0.47057359 -0.04348313  0.04454483]

Using [ 0.46805668  0.29091689 -0.4656586   0.26363562] as initial distribution:
kl-divergence   empirical distribution
0.04601965027   [ 0.07790481 -0.29579551  0.31138016 -0.45748193]
0.87250698684   [-0.26430767  0.32578508 -0.4163628   0.48410916]
0.15913956270   [-0.10058465  0.15307484 -0.36797318 -0.44144324]
0.08849430078   [-0.14201258 -0.22387478 -0.0055356   0.20523729]
0.87447739837   [ 0.16068949 -0.40696394 -0.38444922 -0.06846584]

Using [-0.15540328 -0.3117504   0.4342334  -0.36967832] as initial distribution:
kl-divergence   empirical distribution
0.37095381202   [ 0.48821473  0.04894476  0.21355591 -0.33363249]
0.76438455124   [ 0.49875189 -0.28316116  0.00244861 -0.06103409]
0.23852354213   [-0.24144185  0.22919356 -0.41956833  0.46161728]
0.68577401732   [-0.14448784 -0

### Idea of solution

fill in

### Discussion
fill in